In [ ]:
using Neo4jBolt  
using Dates     

In [ ]:
driver = Neo4jBoltDriver("bolt://localhost:7687", auth=("neo4j", "xxxx"))
sess = session(driver) 

In [ ]:
mutable struct Message
    name::String
    content::String
    mod_date::Date
end

In [ ]:
  
function write_message(message::Message)
    
    currently = Dates.now()
    
    begin_transaction(sess) do tx
        run(tx, 
            "MERGE (m:Message {name: \$name}) " *
            "SET m.content = \$content, m.mod_date = \$mod_date", 
            name=message.name, 
            content=message.content,
            mod_date=currently
        )
    end
    
end


function read_message(name::String)
    
    result = nothing
    begin_transaction(sess) do tx
        result = run(tx, 
            "MATCH (m:Message {name: \$name}) " *
            "RETURN m",
            name=name)
    end
    
    p = value(single(result)).properties
    return Message(p["name"],p["content"],p["mod_date"])
    
end


function link_reply(message1::Message,message2::Message)
    begin_transaction(sess) do tx
        run(tx, 
            "MATCH (m1:Message {name: \$name1}) " *
            "MATCH (m2:Message {name: \$name2}) " *
            "MERGE (m1)-[:REPLY]->(m2) ",
            name1=message1.name,
            name2=message2.name
        )
    end
end

In [ ]:
messageToWrite = Message("My message","Hi, honey, can you get me a drink, please?",Dates.now(Dates.UTC))

write_message(messageToWrite)

In [ ]:
myMessage = read_message("My message")

In [ ]:
herReply = Message("Her reply","Of course I do, my darling. I'd do anything for you...",Dates.now(Dates.UTC))

write_message(herReply)
link_reply(myMessage,herReply)

In [ ]:
close(sess)